<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Capstone Project - Analyzing Student Data</title>
</head>
<body>
    <h1>Capstone Project - Analyzing Student Data</h1>
    <section>
    <p>This data set includes benchmark data for a public school district. The assessment should include student scores in both literacy and math. Student names have been redacted for privacy reasons. School and teacher names have also been changed. A few of my initial hypotheses about the data are: </p>
        <ol>
          <li>Students who recieve progress monitoring after one season, should score higher in the following season.</li>
          <li>Literacy and math scores will be within the same range for students at the same school, in the same classroom. Performance in one subject, however, is not always demonstrative of the school's scores in another subject. i.e. a school that is showing low literacy scores, may also have high math scores</li>
          <li>Classrooms with fewer children will have higher scores.</li>
        </ol>
    </section>
</body>
</html>

<body>
    <h3>Import packages and upload data</h3>
</body>

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip3 install python-csv

  Using cached python-csv-0.0.13.tar.gz (26 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Using cached xmltodict-0.13.0-py2.py3-none-any.whl (10.0 kB)
  Using cached demjson-2.2.4.tar.gz (131 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [1 lines of output]
  error in demjson setup command: use_2to3 is invalid.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


<h3>Importing and Cleaning Data</h3>

In [3]:
aims_23 = pd.read_csv('23_WCSU_aims.csv', dtype = {'Schooluid': 'Int64', 'SSID':'Int64','StudentGrade':'Int64', 'Score':'Int64', 'MeasureGrade':'Int64'})
aims_23.head(20) #data doesn't look as clean as it could might need to be imported again
#some numbers coming in as floats when they should be int. District ID is inaccurate. 
#df = pd.read_csv('file.csv', dtype={'a': 'Int32', 'b': 'Int32'}) <  this helps with the floats
aims_23 = pd.DataFrame(aims_23)
aims_23 = aims_23.drop(columns =['Districtuid','SourceID','DistrictName','Classname','SchoolYear','FormCode','CSEM'])
#is battery score tells us its a composite
aims_23.keys()

Index(['Schooluid', 'SchoolName', 'ClassUid', 'Teacher Last', 'Studentuid',
       'SSID', 'StudentGrade', 'Period', 'Type', 'IsBatteryScore',
       'RelatedForm', 'MeasureName', 'MeasureGrade', 'Score', 'VerticalScore',
       'Errors', 'Accuracy', 'ScaledScore', 'AdministrationDate', 'OffGrade',
       'NormGroup', 'Lexile', 'Quantile', 'FallWinRoi', 'WinSprRoi',
       'FallSprRoi', 'FallWinSgp', 'WinSprSgp', 'FallSprSgp', 'Natl', 'Dist',
       'Sch', 'RiskStatus', 'Probability'],
      dtype='object')

In [3]:
aims_23['SchoolName'].unique()

array(['Brown Bear Middle School', 'Double Dog Elementary School',
       'Whipple Elementary School\n Elementary School',
       'Tan Cat Elementary School\n Elementary School',
       'Blue Bird Elementary School',
       'Jackrabbit Village School\n Elementary School',
       'White Rabbit Elementary School\n Elementary School'], dtype=object)

In [4]:
num_students= aims_23['Studentuid'].nunique()
print(f'In total {num_students} students K-8 were assessed.')

In total 414 students K-8 were assessed.


In [5]:
aims_23['MeasureName'].unique()
aims_23['RelatedForm'].unique()

array(['NSF', 'CA', 'SRF', 'VOC', 'MCF', 'NCF-T', 'READB', 'ORF', 'RC',
       'MATHB', 'M-CAP', 'MFF-T', 'MFF-1D', 'LWSF', 'NCF-P', 'ELB', 'WRF',
       'NWF', 'ENB', 'PS', 'AV', 'QDF', 'PC', 'IS', 'NNF', 'QTF', 'LNF',
       'RC-PM'], dtype=object)

In [4]:
#aims_23[['IsBatteryScore','MeasureName']]
aims_23[['MeasureName','Natl','Sch', 'Dist','RiskStatus']].sample(10)
#I'm not totally sure the accuracy or meaning of the school & district scores. I THINK they are number of incorrect answers

,MeasureName,Natl,Sch,Dist,RiskStatus
2682,Oral Reading Fluency,78.0,NaN,79.0,NaN
11850,Reading,12.0,NaN,18.0,HIGH
3225,Reading Comprehension,77.0,NaN,46.0,NaN
12468,Oral Reading Fluency,26.0,NaN,NaN,NaN
7305,Nonsense Word Fluency,44.0,NaN,55.0,NaN
12425,Number Sense Fluency,92.0,NaN,NaN,NaN
1015,Silent Reading Fluency,94.0,66.0,66.0,NaN
796,Mental Computation Fluency,4.0,12.0,12.0,NaN
5100,Reading,14.0,NaN,26.0,HIGH
5258,Reading Comprehension,95.0,NaN,91.0,NaN


In [7]:
# count_high_risk = aims_23[aims_23.IsBatteryScore == 1]
# count_high_risk = count_high_risk.groupby('')

In [5]:
students_at_risk = aims_23[aims_23.IsBatteryScore == 1]
students_at_risk = students_at_risk[['SchoolName','Studentuid', 'StudentGrade','Period','IsBatteryScore','MeasureName','Score','RiskStatus']]
#print(students_at_risk.dtypes)
students_at_risk
students_at_risk.groupby(['RiskStatus'], as_index=False).nunique()[['Studentuid','RiskStatus']]

,Studentuid,RiskStatus
0,207,HIGH
1,337,LOW
2,205,MODERATE


In [6]:
students_at_high_risk = students_at_risk.groupby(['MeasureName','RiskStatus'],as_index=False).nunique()[['Studentuid','MeasureName','RiskStatus']]
students_at_high_risk = students_at_high_risk[(students_at_high_risk.RiskStatus =='HIGH')] 
students_at_high_risk['percent_of_group'] = (students_at_high_risk['Studentuid']/414).round(decimals=2)
students_at_high_risk.sort_values('percent_of_group', ascending=False).reset_index(drop=True)

,Studentuid,MeasureName,RiskStatus,percent_of_group
0,122,Math,HIGH,0.29
1,105,Reading,HIGH,0.25
2,53,Early Literacy,HIGH,0.13
3,40,Early Numeracy,HIGH,0.10


In [7]:
high_risk_periods = students_at_risk.groupby(['Period','RiskStatus'],as_index=False).nunique()[['Studentuid','Period','RiskStatus']]
high_risk_periods = high_risk_periods[(high_risk_periods.RiskStatus=='HIGH')] 
high_risk_periods

,Studentuid,Period,RiskStatus
0,173,Fall,HIGH
3,138,Spring,HIGH
6,148,Winter,HIGH


In [11]:
grade_level_population = aims_23.groupby(['StudentGrade'],as_index=False).nunique()[['StudentGrade','Studentuid']]
grade_level_population 

school_population = aims_23.groupby(['SchoolName'],as_index=False).nunique()[['SchoolName','Studentuid']]
school_population

school_grade_population = aims_23.groupby(['SchoolName','StudentGrade'],as_index=False).nunique()[['SchoolName','StudentGrade','Studentuid']]
school_grade_population.head()

,SchoolName,StudentGrade,Studentuid
0,Blue Bird Elementary School,1,15
1,Blue Bird Elementary School,2,14
2,Blue Bird Elementary School,3,15
3,Blue Bird Elementary School,4,13
4,Blue Bird Elementary School,5,18


In [8]:
grade_levels_risk = students_at_risk.groupby(['StudentGrade','MeasureName','RiskStatus','Period'],as_index=False).nunique()[['StudentGrade','Studentuid','MeasureName','RiskStatus','Period']]
grade_levels_risk = grade_levels_risk[grade_levels_risk.RiskStatus=='HIGH']
grade_levels_risk = grade_levels_risk[grade_levels_risk.Period=='Spring']
grade_levels_risk_merge = pd.merge(grade_levels_risk,grade_level_population, on='StudentGrade',how='left')
grade_levels_risk_merge['percent_of_pop'] = (grade_levels_risk_merge['Studentuid_x']/grade_levels_risk_merge['Studentuid_y']).round(decimals=2)
grade_levels_risk_merge.sort_values('percent_of_pop', ascending=False).reset_index(drop=True)
#overall Second grade students are at higher risk but this might change if we add in measures

NameError: name 'grade_level_population' is not defined

In [13]:
gls_assessment_risk = students_at_risk.groupby(['StudentGrade','MeasureName','RiskStatus'],as_index=False).nunique()[['StudentGrade','MeasureName','Studentuid','RiskStatus']]
gls_assessment_risk = gls_assessment_risk[gls_assessment_risk.RiskStatus=='HIGH']
gls_assessment_risk_merge = pd.merge(gls_assessment_risk,grade_level_population, on='StudentGrade')
gls_assessment_risk_merge['percent_of_pop'] = (gls_assessment_risk_merge['Studentuid_x']/gls_assessment_risk_merge['Studentuid_y']).round(decimals=2)
gls_assessment_risk_merge.sort_values('percent_of_pop', ascending=False).head()

,StudentGrade,MeasureName,Studentuid_x,RiskStatus,Studentuid_y,percent_of_pop
2,2,Math,25,HIGH,46,0.54
0,1,Early Literacy,30,HIGH,57,0.53
12,7,Math,18,HIGH,34,0.53
14,8,Math,18,HIGH,35,0.51
3,2,Reading,23,HIGH,46,0.50


In [14]:
gl2_assessment_risk = students_at_risk.groupby(['StudentGrade','MeasureName','RiskStatus'],as_index=False).nunique()[['StudentGrade','MeasureName','Studentuid','RiskStatus']]
gl2_assessment_risk = gl2_assessment_risk[gl2_assessment_risk.StudentGrade ==2]
gl2_assessment_risk_merge = pd.merge(gl2_assessment_risk,grade_level_population, on='StudentGrade')
gl2_assessment_risk_merge['percent_of_pop'] = (gl2_assessment_risk_merge['Studentuid_x']/gl2_assessment_risk_merge['Studentuid_y']).round(decimals=2)
gl2_assessment_risk_merge.sort_values('percent_of_pop', ascending=False).head()

,StudentGrade,MeasureName,Studentuid_x,RiskStatus,Studentuid_y,percent_of_pop
1,2,Math,31,LOW,46,0.67
4,2,Reading,30,LOW,46,0.65
0,2,Math,25,HIGH,46,0.54
3,2,Reading,23,HIGH,46,0.50
5,2,Reading,14,MODERATE,46,0.30


In [15]:
gl2_high_risk = students_at_risk.groupby(['StudentGrade','SchoolName','MeasureName','RiskStatus'],as_index=False).nunique()[['StudentGrade','SchoolName','MeasureName','Studentuid','RiskStatus']]
gl2_high_risk = gl2_high_risk[gl2_high_risk.StudentGrade ==2]
gl2_high_risk = gl2_high_risk[gl2_high_risk.RiskStatus == 'HIGH']
gl2_high_risk_merge = pd.merge(gl2_high_risk,grade_level_population, on='StudentGrade')
gl2_high_risk_merge['percent_of_pop'] = (gl2_high_risk_merge['Studentuid_x']/gl2_high_risk_merge['Studentuid_y']).round(decimals=2)
gl2_high_risk_merge.sort_values('percent_of_pop', ascending=False)

,StudentGrade,SchoolName,MeasureName,Studentuid_x,RiskStatus,Studentuid_y,percent_of_pop
6,2,Tan Cat Elementary School\n Elementary School,Math,10,HIGH,46,0.22
0,2,Blue Bird Elementary School,Math,9,HIGH,46,0.20
7,2,Tan Cat Elementary School\n Elementary School,Reading,8,HIGH,46,0.17
3,2,Double Dog Elementary School,Reading,7,HIGH,46,0.15
1,2,Blue Bird Elementary School,Reading,6,HIGH,46,0.13
2,2,Double Dog Elementary School,Math,5,HIGH,46,0.11
4,2,Jackrabbit Village School\n Elementary School,Math,2,HIGH,46,0.04
5,2,Jackrabbit Village School\n Elementary School,Reading,2,HIGH,46,0.04
8,2,Whipple Elementary School\n Elementary School,Reading,1,HIGH,46,0.02
9,2,White Rabbit Elementary School\n Elementary Sc...,Math,1,HIGH,46,0.02


In [16]:
schools_at_risk = students_at_risk.groupby(['SchoolName','RiskStatus'],as_index=False).nunique()[['SchoolName','Studentuid','RiskStatus']]
schools_at_risk = schools_at_risk[schools_at_risk.RiskStatus =='HIGH'] 
schools_at_risk_merge = pd.merge(schools_at_risk,school_population, on='SchoolName', how='left')
schools_at_risk_merge['percent_of_pop'] = (schools_at_risk_merge['Studentuid_x']/schools_at_risk_merge['Studentuid_y']).round(decimals=2)
schools_at_risk_merge.sort_values('percent_of_pop', ascending=False).head()
#brownbear MS has the most students at risk by count & percent of population
#schools_at_risk_merge['Studentuid_x'].sum()

,SchoolName,Studentuid_x,RiskStatus,Studentuid_y,percent_of_pop
1,Brown Bear Middle School,57,HIGH,103,0.55
0,Blue Bird Elementary School,47,HIGH,89,0.53
4,Tan Cat Elementary School\n Elementary School,34,HIGH,68,0.50
2,Double Dog Elementary School,35,HIGH,74,0.47
6,White Rabbit Elementary School\n Elementary Sc...,9,HIGH,19,0.47


In [17]:
schools_measure_risk = students_at_risk.groupby(['SchoolName','MeasureName','RiskStatus'],as_index=False).nunique()[['SchoolName','MeasureName','Studentuid','RiskStatus']]
schools_measure_risk = schools_measure_risk[schools_measure_risk.RiskStatus =='HIGH']

schools_measure_risk_merge = pd.merge(schools_measure_risk,school_population, on='SchoolName', how='left')
schools_measure_risk_merge['percent_of_pop'] = (schools_measure_risk_merge['Studentuid_x']/schools_measure_risk_merge['Studentuid_y']).round(decimals=2)
schools_measure_risk_merge.sort_values('percent_of_pop', ascending=False).head()

,SchoolName,MeasureName,Studentuid_x,RiskStatus,Studentuid_y,percent_of_pop
4,Brown Bear Middle School,Math,54,HIGH,103,0.52
13,Jackrabbit Village School\n Elementary School,Reading,7,HIGH,20,0.35
5,Brown Bear Middle School,Reading,32,HIGH,103,0.31
12,Jackrabbit Village School\n Elementary School,Math,6,HIGH,20,0.30
2,Blue Bird Elementary School,Math,26,HIGH,89,0.29


In [18]:
students_at_risk_detail = students_at_risk.groupby(['StudentGrade','MeasureName','RiskStatus'],
                                                   as_index=False).nunique()[['Studentuid','StudentGrade','MeasureName','RiskStatus']]
students_at_risk_detail[students_at_risk_detail.RiskStatus == 'HIGH']

,Studentuid,StudentGrade,MeasureName,RiskStatus
0,30,1,Early Literacy,HIGH
3,27,1,Early Numeracy,HIGH
6,25,2,Math,HIGH
9,23,2,Reading,HIGH
12,20,3,Math,HIGH
15,21,3,Reading,HIGH
18,15,4,Math,HIGH
21,17,4,Reading,HIGH
24,6,5,Math,HIGH
27,11,5,Reading,HIGH


<h4>Naming Conventions</h4>

<label>naming convention</label>
<ol>
          <li>Grade Levels: i.e. gl1 means first grade, glk_2 = grade levels K-2</li>
          <li>Schools: i.e. bbe means blue bird elementary, bbe_gl1 = first grade at blue bird elementary </li>
          <li>Literacy components (may vary by grade level) = lit or lit_lnf for letter naming fluency or bb3_gl1_lit_lnf</li>
          <li>Math components (may vary by grade level) = math or gl8_math</li>
          <li>Classrooms/Teachers= use class id, bbe_gl1_35583_lit (might be a little redundant with school and grade level)</li>
          <li>Students - sid+student id i.e. sid_13498</li>
          <li>Year- two digit '23' for SY2022-2023</li>
        </ol>

<h1>battery scores</h1>

<h3>math</h3>

In [19]:
aims_23_math= aims_23[(aims_23.RelatedForm =='ENB')|(aims_23.RelatedForm =="MATHB")]
aims_23_math = aims_23_math.drop(columns = ['SSID','Type','VerticalScore','Errors', 'Accuracy', 'ScaledScore', 'AdministrationDate', 'OffGrade','NormGroup', 'Lexile', 'Quantile', 'FallWinRoi', 'WinSprRoi','FallSprRoi', 'FallWinSgp', 'WinSprSgp', 'FallSprSgp', 'Natl', 'Dist','Sch', 'RiskStatus', 'Probability'])
#aims_23_math= aims_23_math.groupby(['MeasureName','StudentGrade','Period'])
#aims_23_math

<h3>lit</h3>

In [20]:
aims_23_lit= aims_23[(aims_23.RelatedForm == 'ELB')|(aims_23.RelatedForm=="READB")]
aims_23_lit = aims_23_lit.drop(columns = ['SSID','Type','VerticalScore','Errors', 'Accuracy', 'ScaledScore', 'AdministrationDate', 'OffGrade','NormGroup', 'Lexile', 'Quantile', 'FallWinRoi', 'WinSprRoi','FallSprRoi', 'FallWinSgp', 'WinSprSgp', 'FallSprSgp', 'Probability'])
# aims_23_lit= aims_23.loc[aims_23_lit]
# aims_23_lit= aims_23_lit.groupby(by=['MeasureName','StudentGrade','Period','Score']).mean().round().reset_index()
#aims_23_lit[['Natl','Sch', 'Dist','RiskStatus']]
aims_23_lit= aims_23_lit.fillna(0)
aims_23_lit.head()


,Schooluid,SchoolName,ClassUid,Teacher Last,Studentuid,StudentGrade,Period,IsBatteryScore,RelatedForm,MeasureName,MeasureGrade,Score,Natl,Dist,Sch,RiskStatus
8,234,Brown Bear Middle School,35583,Sanderson,11587,8,Spring,1,READB,Reading,8,475,22.0,16.0,16.0,HIGH
17,234,Brown Bear Middle School,35583,Sanderson,11587,8,Winter,1,READB,Reading,8,529,54.0,31.0,31.0,LOW
27,234,Brown Bear Middle School,35553,Henry,11877,8,Spring,1,READB,Reading,8,574,77.0,73.0,73.0,LOW
39,234,Brown Bear Middle School,35553,Henry,11877,8,Winter,1,READB,Reading,8,571,78.0,54.0,54.0,LOW
40,234,Brown Bear Middle School,35553,Henry,11877,8,Fall,1,READB,Reading,8,571,78.0,66.0,66.0,LOW


<h3>norms</h3>

In [30]:
aims_norms = pd.read_csv('aims_norms.csv')
aims_norms = pd.DataFrame(aims_norms) #noticing a few issues with my column names
aims_norms.rename(columns=
                  {'Grade Level':'StudentGrade','Assessment Category':'AssessmentCat', 
                   'Assessment Name':'MeasureName',
                   'Season':'Period','Standard Deviation':'SD',
                   '10th Percentile':'%10th', '50th Percentile':'%50th',
                   '99th Percentile':'%99th'}, inplace=True)
aims_norms.keys()
aims_norms.to_csv('aims_norms_clean.csv')

<label><strong>high level view</strong></label>

In [29]:
#aims_23_math.merge(aims_norms, on = 'RelatedForm', how = 'outer')
aims_norms_math = aims_norms[(aims_norms.RelatedForm == 'ENB')|(aims_norms.RelatedForm=="MATHB")]
aims_norms_lit = aims_norms[(aims_norms.RelatedForm == 'ELB')|(aims_norms.RelatedForm=="READB")]
aims_norms_math = aims_norms_math.drop(columns=['Unique Identifier', 'AssessmentCat', 'Mean', 'SD']).reset_index()
aims_norms_math = aims_norms_math.drop(columns=['index'])
aims_norms_math['StudentGrade'] = aims_norms_math['StudentGrade'].astype('string')
aims_norms_math[['%10th','%50th','%99th']] = aims_norms_math[['%10th','%50th','%99th']].apply(pd.to_numeric)
#print(aims_norms_math.dtypes)
aims_norms_math.to_csv('aims_norms_math')


In [23]:
aims_norms_math_filtered = aims_norms_math.drop([0,1,2])
aims_norms_math_gl_averages=aims_norms_math_filtered.groupby(['StudentGrade'], as_index=False).agg(
    avg_10th= ('%10th', np.mean),
    avg_50th= ('%50th', np.mean),
    avg_99th= ('%99th', np.mean)).round()
aims_norms_math_gl_averages
print(aims_norms_math_gl_averages.dtypes)

StudentGrade    string[python]
avg_10th               float64
avg_50th               float64
avg_99th               float64
dtype: object


C:\Users\WCSU User\AppData\Local\Temp\ipykernel_21228\2555228952.py:2: FutureWarning: The provided callable <function mean at 0x0000021A9A7E3380> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  aims_norms_math_gl_averages=aims_norms_math_filtered.groupby(['StudentGrade'], as_index=False).agg(


In [24]:
aims_23_math_gls= aims_23_math.groupby(['StudentGrade'], as_index=False).agg(
        num_students = ('Studentuid', np.count_nonzero),
        avg_math_score = ('Score', np.mean)).round()

aims_23_math_gls['StudentGrade'] = aims_23_math_gls['StudentGrade'].astype('string')
# # # #aims_23_math_gls.['StudentGrade'] = aims_23_math_gls['StudentGrade'].astype('Int64')
aims_23_math_gls = pd.merge(aims_23_math_gls,aims_norms_math_gl_averages, how='outer')
# aims_23_math_gls['avg_score'] = aims_23_math_gls['avg_score'].fillna(0)
aims_23_math_gls['avg_math_score'] = aims_23_math_gls['avg_math_score'].astype('float64')
print(aims_23_math_gls.dtypes)
aims_23_math_gls
# aims_23_math_gls.groupby(['StudentGrade']).mean(numeric_only=True)[['num_students','avg_score','%10th','%50th','%99th']].round()
# aims_23_math_gls

StudentGrade      string[python]
num_students               int64
avg_math_score           float64
avg_10th                 float64
avg_50th                 float64
avg_99th                 float64
dtype: object


C:\Users\WCSU User\AppData\Local\Temp\ipykernel_21228\616288781.py:1: FutureWarning: The provided callable <function mean at 0x0000021A9A7E3380> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  aims_23_math_gls= aims_23_math.groupby(['StudentGrade'], as_index=False).agg(


,StudentGrade,num_students,avg_math_score,avg_10th,avg_50th,avg_99th
0,1,162,57.0,37.0,61.0,125.0
1,2,138,173.0,135.0,180.0,303.0
2,3,146,206.0,159.0,204.0,326.0
3,4,135,219.0,172.0,213.0,323.0
4,5,164,248.0,184.0,226.0,341.0
5,6,130,228.0,186.0,230.0,350.0
6,7,99,220.0,184.0,229.0,350.0
7,8,104,228.0,190.0,234.0,357.0


In [25]:
conditions = [
    (aims_23_math_gls['avg_math_score'] <= aims_23_math_gls['avg_10th']),
    (aims_23_math_gls['avg_math_score'] >= aims_23_math_gls['avg_10th']) &
    (aims_23_math_gls['avg_math_score'] <= aims_23_math_gls['avg_50th']),
    (aims_23_math_gls['avg_math_score'] >= aims_23_math_gls['avg_50th'])
]

values = ['tier 1','tier 2','tier 3']
aims_23_math_gls['tier'] = np.select(conditions, values)

aims_23_math_gls['percentile'] = (aims_23_math_gls['avg_math_score']/aims_23_math_gls['avg_99th']).round(decimals=2)
aims_23_math_gls

,StudentGrade,num_students,avg_math_score,avg_10th,avg_50th,avg_99th,tier,percentile
0,1,162,57.0,37.0,61.0,125.0,tier 2,0.46
1,2,138,173.0,135.0,180.0,303.0,tier 2,0.57
2,3,146,206.0,159.0,204.0,326.0,tier 3,0.63
3,4,135,219.0,172.0,213.0,323.0,tier 3,0.68
4,5,164,248.0,184.0,226.0,341.0,tier 3,0.73
5,6,130,228.0,186.0,230.0,350.0,tier 2,0.65
6,7,99,220.0,184.0,229.0,350.0,tier 2,0.63
7,8,104,228.0,190.0,234.0,357.0,tier 2,0.64


<h3>Matplotlib??</h3>

In [36]:
import csv
from itertools import groupby
aims_23_bat = aims_23[(aims_23.IsBatteryScore == 1)]
aims_23_bat = aims_23_bat.drop(columns=['VerticalScore','Errors', 'Accuracy', 'ScaledScore', 'AdministrationDate', 'OffGrade',
       'NormGroup', 'Lexile', 'Quantile', 'FallWinRoi', 'WinSprRoi',
       'FallSprRoi', 'FallWinSgp', 'WinSprSgp', 'FallSprSgp']).reset_index()

aims_23_battery = aims_23_bat.to_csv('aims_23_bat.csv')
battery_data='aims_23_bat.csv'
def read_csv(filename):
    with open(filename, 'r') as f:
        data = list(csv.read(f))
        return data
    
battery_scores = read_csv('aims_23_bat.csv')
#not working :(

AttributeError: module 'csv' has no attribute 'read'

In [18]:

colors ={
    1: #AD343E,
    2: #474747,
    3: #845EC2,
    4: #D65DB1,
    5: #FF6F91,
    6: #FF9671,
    7: #FFC75F,
    8: #F9F871
}
for grade, num_students in groupby (aims_23_lit, lambda x:x[5]):
    grade_level= list(group)
    score = [int(aims_23_lit[11] fot student in grade_level)]

SyntaxError: positional argument follows keyword argument (291308496.py, line 2)

In [26]:

# aims_23_math_gls_suavgs = aims_23_math_gls_suavgs.groupby(['StudentGrade']).agg(
#     num_students= ('StudentGrade')
#     avg_gl_score= ('avg_score', np.mean),
#     avg_10th= ('%10th', np.mean),
#     avg_50th= ('%50th', np.mean),
#     avg_99th= ('%99th', np.mean)).round()


# avg_gl_score_conditions = [
#     (aims_23_math_gls_suavgs['avg_gl_score'] <= aims_23_math_gls_suavgs['avg_10th']),
#     (aims_23_math_gls_suavgs['avg_gl_score'] >= aims_23_math_gls_suavgs['avg_10th']) &
#     (aims_23_math_gls_suavgs['avg_gl_score'] <= aims_23_math_gls_suavgs['avg_50th']),
#     (aims_23_math_gls_suavgs['avg_gl_score'] >= aims_23_math_gls_suavgs['avg_50th'])
# ]

# avg_gl_score_values= ['tier 1','tier 2','tier 3']

# aims_23_math_gls_suavgs['tier'] = np.select(avg_gl_score_conditions, avg_gl_score_values)
# aims_23_math_gls_suavgs

<h3>Asking Questions</h3>
<p>Rather than looking to answer the hypotheses I made, I'm going to ask questions to help me organize my thinking</p>
<p><strong>Q1:Overall, which area- math or literacy- is the SU doing best in?</strong></p>

In [28]:
# aims_23_SU_math = aims_23_SU_math.agg(
#     num_students = ('num_students', np.sum),
#     avg_math_score = ('avg_math_score', np.mean),
#     avg_10th = ('avg_10th',np.mean),
#     avg_50th = ('avg_50th', np.mean),
#     avg_99th = ('avg_99th', np.mean),
#     percentile = ('percentile', np.mean)).round(decimals=2)
# aims_23_SU_math.describe()
# aims_23_SU_math #if the math is accurate it shows that our students are, overall, above the 50th percentile in math


<h3>OLD: Playing with grouping</h3>

In [ ]:
#High level look at JUST Math and Literacy Composite Scores
aims_23_gls = np.where((aims_23['IsBatteryScore'] == 1))
aims_23_gls = aims_23.loc[aims_23_gls]

# def make_grade_tables():
#     for i in range(1,9):
#         x = aims_23_gls[aims_23_gls['StudentGrade'] == i]
#         x = f'aims_23_gls{i}'
#         return x

# make_grade_tables()
#do all grade levels, bring this into R to compare with normed scores

Score
StudentGrade MeasureName Period       
2            Math        Fall    150.0
                         Spring  192.0
                         Winter  177.0
             Reading     Fall    314.0
                         Spring  382.0
                         Winter  360.0

In [ ]:
aims_23_gl2 = aims_23_gls[aims_23_gls['StudentGrade'] == 2]
aims_23_gl2 = aims_23_gl2.groupby(by=['StudentGrade','MeasureName','Period']).aggregate({'Score':'mean'}).round()
aims_23_gl2 

In [ ]:
aims_23_gls.keys()
aims_23_gls = aims_23_gls.drop(columns = ['SSID','VerticalScore', 'Errors', 'Accuracy','ScaledScore', 'AdministrationDate', 'OffGrade', 'NormGroup', 'Lexile',
       'Quantile', 'FallWinRoi', 'WinSprRoi', 'FallSprRoi', 'FallWinSgp', 'WinSprSgp', 'FallSprSgp','Probability'])
aims_23_gls 

,Schooluid,SchoolName,ClassUid,Teacher Last,Studentuid,StudentGrade,Period,Type,IsBatteryScore,RelatedForm,MeasureName,MeasureGrade,Score,Natl,Dist,Sch,RiskStatus,AssessmentType
8,234,Brown Bear Middle School,35583,Sanderson,11587,8,Spring,BM,1,READB,Reading,8,475,22.0,16.0,16.0,HIGH,literacy
11,234,Brown Bear Middle School,35583,Sanderson,11587,8,Spring,BM,1,MATHB,Math,8,214,29.0,39.0,39.0,MODERATE,math
14,234,Brown Bear Middle School,35583,Sanderson,11587,8,Winter,BM,1,MATHB,Math,8,202,15.0,21.0,21.0,HIGH,math
17,234,Brown Bear Middle School,35583,Sanderson,11587,8,Winter,BM,1,READB,Reading,8,529,54.0,31.0,31.0,LOW,literacy
27,234,Brown Bear Middle School,35553,Henry,11877,8,Spring,BM,1,READB,Reading,8,574,77.0,73.0,73.0,LOW,literacy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11972,<NA>,Tan Cat Elementary School\n Elementary School,34008,Richardson,13521,2,Spring,BM,1,MATHB,Math,2,227,82.0,79.0,NaN,LOW,math
11979,<NA>,Tan Cat Elementary School\n Elementary School,34008,Richardson,13521,2,Spring,BM,1,READB,Reading,2,439,80.0,83.0,NaN,LOW,literacy
11981,<NA>,Tan Cat Elementary School\n Elementary School,34008,Richardson,13521,2,Winter,BM,1,READB,Reading,2,390,62.0,73.0,NaN,LOW,literacy
11985,<NA>,Tan Cat Elementary School\n Elementary School,33944,Paugh,13530,1,Spring,BM,1,ELB,Early Literacy,1,111,84.0,85.0,NaN,LOW,literacy


In [ ]:
aims_23_gls.merge(aims_norms, on = 'RelatedForm', how = 'outer') #this isn't right

,Schooluid,SchoolName,ClassUid,Teacher Last,Studentuid,StudentGrade,Period,Type,RelatedForm,MeasureName,...,Unique Identifier,Grade Level,Assessment Category,Assessment Name,Season,Mean,Standard Deviation,10th Percentile,50th Percentile,99th Percentile
0,234,Brown Bear Middle School,35583.0,Sanderson,11587.0,8,Spring,BM,READB,Reading,...,Lit-2-R-CS-Fa,2,Literacy,Reading,Fall,339.1,64.2,256.9,339.1,563.16
1,234,Brown Bear Middle School,35583.0,Sanderson,11587.0,8,Spring,BM,READB,Reading,...,Lit-2-R-CS-Wi,2,Literacy,Reading,Winter,368.5,58.1,294.1,368.5,571.27
2,234,Brown Bear Middle School,35583.0,Sanderson,11587.0,8,Spring,BM,READB,Reading,...,Lit-2-R-CS-Sp,2,Literacy,Reading,Spring,392.5,57.5,318.9,392.5,593.18
3,234,Brown Bear Middle School,35583.0,Sanderson,11587.0,8,Spring,BM,READB,Reading,...,Lit-3-R-CS-Fa,3,Literacy,Reading,Fall,391.9,59.8,315.4,391.9,600.6
4,234,Brown Bear Middle School,35583.0,Sanderson,11587.0,8,Spring,BM,READB,Reading,...,Lit-3-R-CS-Wi,3,Literacy,Reading,Winter,411,57.4,337.5,411,611.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41932,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,SRF,NaN,...,Lit-7-SRF-Wi,7,Literacy,Silent Reading Fluency,Winter,166.1,50.6,101.3,166.1,342.69
41933,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,SRF,NaN,...,Lit-7-SRF-Sp,7,Literacy,Silent Reading Fluency,Spring,171,44.7,113.8,171,327
41934,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,SRF,NaN,...,Lit-8-SRF-Fa,8,Literacy,Silent Reading Fluency,Fall,153.5,46.4,94.1,153.5,315.44
41935,<NA>,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,SRF,NaN,...,Lit-8-SRF-Wi,8,Literacy,Silent Reading Fluency,Winter,161.2,44.1,104.8,161.2,315.11


In [ ]:
#Middle School --> Likely has the least amount of data and possibly some of the lower scores

bbms_aims_23 = aims_23[aims_23['SchoolName'] == 'Brown Bear Middle School']
bbms_aims_23.drop(columns=['Schooluid','SchoolName'])

#breaking data up into grade levels
bbms_aims_gl6 = bbms_aims_23[bbms_aims_23['StudentGrade']== 6]
bbms_aims_gl7 = bbms_aims_23[bbms_aims_23['StudentGrade']== 7]
bbms_aims_gl8 = bbms_aims_23[bbms_aims_23['StudentGrade']== 8]

bbms_aims_gl6= bbms_aims_gl6.groupby(by=['MeasureName','ClassUid']).aggregate({'Score':'mean','Natl':'mean', 'Dist':'mean','Sch':'mean'}).round(decimals=2)
bbms_aims_gl6.describe().round(decimals=2)

#ROI = Rate of Improvement. It looks like classroom 35583 saw an almost 50% ROI from winter to spring!
#max classroom avg = 602 , min classroom avg = 2.67... seems incorrect?
bbms_aims_gl8.groupby(by=['StudentGrade','MeasureName','Period']).aggregate({'Score':'mean'}).round()

Score
StudentGrade MeasureName                         Period       
8            Concepts & Applications             Fall     12.0
                                                 Spring   12.0
                                                 Winter   12.0
             Math                                Fall    222.0
                                                 Spring  234.0
                                                 Winter  227.0
             Mathematics Concepts & Applications Fall      6.0
                                                 Winter    8.0
             Mental Computation Fluency          Fall      9.0
                                                 Spring   11.0
                                                 Winter    8.0
             Number Comparison Fluency-Triads    Fall     14.0
                                                 Spring   15.0
                                                 Winter   14.0
             Number Sense Fluency                Fall     23.0
                                                 Spring   25.0
                                                 Winter   22.0
             Oral Reading Fluency                Fall    136.0
                                                 Spring  132.0
                                                 Winter  138.0
             Reading                             Fall    541.0
                                                 Spring  541.0
                                                 Winter  555.0
             Reading Comprehension               Fall     13.0
                                                 Spring   13.0
                                                 Winter   14.0
             Silent Reading Fluency              Fall    210.0
                                                 Spring  211.0
                                                 Winter  212.0
             Vocabulary                          Fall     15.0
                                                 Spring   15.0
                                                 Winter   16.0

<h3>Calculating Tiers</h3>
<p>Multi-tiered Systems of Support (MTSS) is a system that divides student scores into three tiers based on assessment</p>

In [ ]:
#create formula for tiers
#51-99 percentile,11th-50th,10th and below 
#rework into formula once categories are established in formula above